In [1]:
from Rings import Zomega, Ztau, Exact_U
from norm_equation import easy_factor, easy_solveable, solve_norm_equation
from exact_synthesize import exact_synthesize, exact_synthesize_weave
from Circuits import FTCircuit, Braid
from math import sqrt, ceil, pi, log, sin, cos, tan, acos
import cmath
import random
import numpy as np
from toqito.random import random_unitary

`approx_real` approximates a real number x to precision \(\tau^{n-1}(1 - \tau^n)\)

In [2]:
FibArray = [1, 1]
 
def Fib(n):
    if n<0:
        print("Incorrect input")
    elif n<= len(FibArray):
        return FibArray[n-1]
    else:
        temp_fib = Fib(n-1)+Fib(n-2)
        FibArray.append(temp_fib)
        return temp_fib    

In [3]:
def approx_real(x,n):
    p, q = Fib(n), Fib(n+1)
    u, v = (-1)**(n+1)*Fib(n), (-1)**n * Fib(n-1)
    c = round(x*q)
    a = c*v + p * round(c*u/q)
    b = c*u - q * round(c*u/q)

    return Ztau(a, b)

In [4]:
print(approx_real(pi, 20))

(-317 + 518τ)


`random_sample` generates an element of Z[omega] approximating exp(-i theta/2) to within precision epsilon

In [5]:
phi = 0.5 * (sqrt(5) + 1)
tau = 0.5 * (sqrt(5) - 1)

In [6]:
def random_sample(theta, epsilon, r):
    C = sqrt(phi/(4*r))
    m = ceil(log(C*epsilon*r, tau)) + 1
    N = ceil(phi**m)

    y_min = r*phi**m *(sin(theta) - epsilon*(sqrt(4-epsilon**2)*cos(theta) + epsilon*sin(theta))/2)
    y_max = r*phi**m *(sin(theta) + epsilon*(sqrt(4-epsilon**2)*cos(theta) - epsilon*sin(theta))/2)
    x_max = r*phi**m *((1 - epsilon**2/2)*cos(theta) - epsilon*sqrt(1 - epsilon**2/4)*sin(theta))
    x_c = x_max - r*epsilon**2*phi**m/(4*cos(theta))

    j = random.randint(1, N-1)
    y = y_min + j*(y_max - y_min)/N
    a_y = approx_real(y/sqrt(2-tau), m)
    x = x_c - (a_y.value()*sqrt(2-tau) - y_min)*tan(theta)
    a_x = approx_real(x,m)

    return Zomega.fromTau(a_x) + Zomega(-1,2,-1,1)*Zomega.fromTau(a_y)


In [7]:
print(random_sample(pi/4, 1/100, 5).value())

427.124611797498 + 433.800981547744*I


In [8]:
def compile_R_rotation(angle, epsilon):
    C = sqrt(phi/4)
    m = ceil(log(C*epsilon, tau)) + 1
    theta = 0
    for k in range(10):       
        adjusted_angle = (-angle/2 - pi*k/5) - ((-angle/2 - pi*k/5)//(2*pi)) * (2*pi)
        if adjusted_angle >= 0 and adjusted_angle <= pi/5:
            theta = adjusted_angle
            break
    
    not_found, u, v = True, 0, 0

    while not_found:
        u0 = random_sample(theta, epsilon, 1)
        phi_t = Ztau(1, 1)
        xi = phi_t*(phi_t**(2*m) - Ztau.fromOmega(u0*u0.star()))
        factors = easy_factor(xi)
        if easy_solveable(factors):
            not_found = False
            #print("found one!")
            #print("u0 angle: " + str(cmath.phase(u0.value())))
            u = Zomega(0,1,0,0)**k * Zomega.fromTau(Ztau(0,1))**m * u0
            #print("norm solution: " + str(solve_norm_equation(xi)))
            v = Zomega.fromTau(Ztau(0,1))**m * solve_norm_equation(xi)
    
    print("u, v = " + str(u) + ", " + str(v))
    #print("u,v value: " + str(u.value()) + ", " + str(v.value()))
    #print("u angle: " + str(cmath.phase(u.value())))
    print("distance = " + str(sqrt(1 - 0.5 * abs((u.value()*cmath.exp(1j*angle/2) + (u.star().value()*cmath.exp(-1j*angle/2)))))))
    #print("-----------------------")
    #print(Exact_U(u,v,0).value())
    Circuit = exact_synthesize(Exact_U(u, v, 0))

    return Circuit

In [9]:
def compile_RX_rotation(angle, epsilon):
    r = sqrt(phi)
    C = sqrt(phi/(4*r))
    m = ceil(log(C*epsilon*r, tau)) + 1
    theta = 0
    for k in range(10):
        adjusted_angle = (angle/2 + pi/2 - pi*k/5) - ((angle/2 + pi/2 - pi*k/5)//(2*pi)) * (2*pi)
        if adjusted_angle >= 0 and adjusted_angle <= pi/5:
            theta = adjusted_angle
            break
    
    not_found, u, v = True, 0, 0

    while not_found:
        u0 = random_sample(theta, epsilon, r)
        tau_t = Ztau(0,1)
        phi_t = Ztau(1,1)
        xi = phi_t**(2*m) - tau_t * Ztau.fromOmega(u0*u0.star())
        factors = easy_factor(xi)
        if easy_solveable(factors):
            not_found = False
            #print("found one!")
            #print(k)
            v = Zomega(0,1,0,0)**k * Zomega.fromTau(Ztau(0,1))**m * u0
            #print("norm solution: " + str(solve_norm_equation(xi)))
            u = Zomega.fromTau(Ztau(0,1))**m * solve_norm_equation(xi)
    
    v = Zomega(-1,0,0,0) * v.star()
    ex = Exact_U(u, v, 0)
    print(ex.value())
    #print("-----------------------")
    Circuit = exact_synthesize(ex)

    return Circuit

In [10]:
def Rz(angle):
    return np.array([
        [cmath.exp(-0.5j*angle), 0],
        [0, cmath.exp(0.5j*angle)]
    ])

def Pauli_X():
    return np.array([
        [0, 1],
        [1, 0]
    ])

def F():
    return np.array([
        [tau, sqrt(tau)],
        [sqrt(tau), -1*tau]
    ])

In [11]:
# I think this algorithm gives a distance of sqrt(epsilon)

'''
angle = pi/2
print(np.matmul(Rz(angle),Pauli_X()))
FT = compile_RX_rotation(angle, 0.0001)
print(FT)
#print(FT.numerical_approximate())
#print(FT.toBraid())
#print(FT.toBraid().numerical_approximate())

angle = 6*pi/7
print(Rz(angle))
FT = compile_R_rotation(angle, 0.0001)
print(FT.numerical_approximate())
print(FT.toBraid())
print(FT.toBraid().numerical_approximate())
'''

'\nangle = pi/2\nprint(np.matmul(Rz(angle),Pauli_X()))\nFT = compile_RX_rotation(angle, 0.0001)\nprint(FT)\n#print(FT.numerical_approximate())\n#print(FT.toBraid())\n#print(FT.toBraid().numerical_approximate())\n\nangle = 6*pi/7\nprint(Rz(angle))\nFT = compile_R_rotation(angle, 0.0001)\nprint(FT.numerical_approximate())\nprint(FT.toBraid())\nprint(FT.toBraid().numerical_approximate())\n'

In [12]:
print(Braid.evaluate_braid("2"))

[[-0.5       +3.63271264e-01j -0.24293414-7.47674391e-01j]
 [-0.24293414-7.47674391e-01j -0.61803399+1.54317926e-16j]]


In [13]:
def decompose_unitary(U):
    # decompose U into e^i*delta Rz(alpha)*F*Rz(beta)*F*Rz(gamma) or e^id Rz(a)*F*Rz(b)*F*Rz(c) X
    X = False
    a, b, c, d = U[0,0], U[0,1], U[1,0], U[1,1]
    if abs(a) < 0.5:
        X = True
        a, b, c, d = U[0,1], U[0,0], U[1,1], U[1,0]
    
    delta = 0.5 * cmath.phase(a*d - b*c)

    a, b, c, d = cmath.exp(-1j * delta)*a, cmath.exp(-1j * delta)*b, cmath.exp(-1j * delta)*c, cmath.exp(-1j * delta)*d, 
    
    beta = acos(1 - abs(c)**2/(2*tau**3))
    alpha = -(cmath.phase(a) - cmath.phase(cmath.exp(1j*beta) + tau)
              - cmath.phase(c) + cmath.phase(1 - cmath.exp(1j*beta)))
    gamma = -2*(cmath.phase(a) - cmath.phase(cmath.exp(1j*beta) + tau)) - alpha - beta
    
    return (alpha, beta, gamma, delta, X)


In [14]:
def compile_unitary(U, epsilon):
    # break U into rotations and then compile them individually
    # has precision epsilon^(1/6)
    (alpha, beta, gamma, delta, X) = decompose_unitary(U)
    #print(cmath.exp(1j*delta)*np.matmul(Rz(alpha), np.matmul(F(), np.matmul(Rz(beta), np.matmul(F(), Rz(gamma))))))
    A = compile_R_rotation(alpha, epsilon)
    B = compile_R_rotation(beta, epsilon)
    if X:
        C = compile_RX_rotation(gamma, epsilon)
    else:
        C = compile_R_rotation(gamma, epsilon)


    # debug block
    A_num, B_num, C_num = A.numerical_approximate(), B.numerical_approximate(), C.numerical_approximate()
    estimate = cmath.exp(delta*pi*1j) * np.matmul(A_num, np.matmul(F(), np.matmul(B_num, np.matmul(F(), C_num))))
    print(abs(estimate[0]))

    F_circ = FTCircuit()
    F_circ.join("F", 1)

    A_braid, B_braid, C_braid, F_braid = A.toBraid(), B.toBraid(), C.toBraid(), F_circ.toBraid()

    total_braid = A_braid + F_braid + B_braid + F_braid + C_braid
    total_braid.addPhase(delta)

    return total_braid

In [15]:
'''
U = random_unitary(2)
print(U)
print(decompose_unitary(U))
braid = compile_unitary(U, 0.0001)
print(braid)
print(braid.numerical_approximate())
'''

'\nU = random_unitary(2)\nprint(U)\nprint(decompose_unitary(U))\nbraid = compile_unitary(U, 0.0001)\nprint(braid)\nprint(braid.numerical_approximate())\n'

In [16]:
def compile_R_weave(angle, epsilon):
    weave = False
    while not weave:
        C = sqrt(phi/4)
        m = ceil(log(C*epsilon, tau)) + 1
        theta = 0
        for k in range(10):       
            adjusted_angle = (-angle/2 - pi*k/5) - ((-angle/2 - pi*k/5)//(2*pi)) * (2*pi)
            if adjusted_angle >= 0 and adjusted_angle <= pi/5:
                theta = adjusted_angle
                break
        
        not_found, u, v = True, 0, 0

        while not_found:
            u0 = random_sample(theta, epsilon, 1)
            phi_t = Ztau(1, 1)
            xi = phi_t*(phi_t**(2*m) - Ztau.fromOmega(u0*u0.star()))
            factors = easy_factor(xi)
            if easy_solveable(factors):
                not_found = False
                u = Zomega(0,1,0,0)**k * Zomega.fromTau(Ztau(0,1))**m * u0
                v = Zomega.fromTau(Ztau(0,1))**m * solve_norm_equation(xi)
        
        print("u, v = " + str(u) + ", " + str(v))
        print("distance = " + str(sqrt(1 - 0.5 * abs((u.value()*cmath.exp(1j*angle/2) + (u.star().value()*cmath.exp(-1j*angle/2)))))))
        (Circuit, Remainder) = exact_synthesize_weave(Exact_U(u, v, 0))
        print(Remainder)
        if len(Remainder.circuit) == 0:
            weave = True

    return Circuit

In [18]:
angle = pi/2
print(Rz(angle))
FT = compile_R_weave(angle, 0.001)
print(FT.toBraid())
print(FT.toBraid().numerical_approximate())

[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]
u, v = (2249115 + 6336893ω + -13892452ω² + 9976036ω³), (-703335 + 8378303ω + -12418359ω² + 7240283ω³)
distance = 0.02845691259788213
(-1 + 0ω + 0ω² + 0ω³)(T)(F)(T^2)
u, v = (-2385101 + 14079595ω + -18922088ω² + 10220419ω³), (4305637 + -13421820ω + 14750294ω² + -6455153ω³)
distance = 0.028456610306363157
(-1 + 1ω + -1ω² + 1ω³)(T^9)(F)(T^5)
u, v = (6172917 + -10065679ω + 6298622ω² + -77691ω³), (2805027 + 404960ω + -5193868ω² + 4943589ω³)
distance = 0.02845614289624493
(0 + 0ω + -1ω² + 0ω³)(T^9)(F)(T^8)
u, v = (4491503 + -4490573ω + -1504ω² + 2776830ω³), (1220844 + 1736775ω + -4785528ω² + 3712142ω³)
distance = 0.028456279806939676
(-1 + 1ω + -1ω² + 1ω³)(T^2)(F)(T)
u, v = (5220471 + -3426083ω + -2903380ω² + 5020815ω³), (-4310148 + 8010205ω + -5986818ω² + 1036239ω³)
distance = 0.028455707751814805
(-1 + 0ω + 0ω² + 0ω³)(T)(F)(T^8)
u, v = (-324614 + 13747644ω + -21718918ω² + 13222406ω³), (-94132

KeyboardInterrupt: 